<a href="https://colab.research.google.com/github/pankajrana1111/Image-Classification/blob/pankajrana1111-patch-1/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#getting repository from github
!git clone https://github.com/pankajrana1111/Image-Classification.git

In [ ]:
!pip install patool
import patoolib
patoolib.extract_archive("Image-Classification/fashion1.rar", outdir="Image-Classification")
patoolib.extract_archive("Image-Classification/fashion2.rar", outdir="Image-Classification")
patoolib.extract_archive("Image-Classification/fashion3.rar", outdir="Image-Classification")
patoolib.extract_archive("Image-Classification/fashion4.rar", outdir="Image-Classification")
patoolib.extract_archive("Image-Classification/fashion5.rar", outdir="Image-Classification")
patoolib.extract_archive("Image-Classification/fashion6.rar", outdir="Image-Classification")

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data=pd.read_csv("Image-Classification/train.csv")
data.head()

Now Here I see that the files coloumn contains the locations of the images and images coloum contains the name of the image files. Further the target coloum contains the category of the image.

Expolatory Data Analysis

In [ ]:
data.shape

In [ ]:
##plotting the no. of categories
y_labels=data.target.unique()
counts=[data.target.tolist().count(i) for i in y_labels]
plt.barh(y_labels,counts)
plt.show()

In the plot we observed that the no. of images in each category are alomost equal and hence we have a balanced dataset.

Creating Predictor Matrix out of images

In [10]:
import cv2

In [26]:
##creating the lists of image locations
image_loc1=[os.path.join("Image-Classification/fashion1",img) for img in os.listdir("Image-Classification/fashion1")]
image_loc2=[os.path.join("Image-Classification/fashion2",img) for img in os.listdir("Image-Classification/fashion2")]
image_loc3=[os.path.join("Image-Classification/fashion3",img) for img in os.listdir("Image-Classification/fashion3")]
image_loc4=[os.path.join("Image-Classification/fashion4",img) for img in os.listdir("Image-Classification/fashion4")]
image_loc5=[os.path.join("Image-Classification/fashion5",img) for img in os.listdir("Image-Classification/fashion5")]
image_loc6=[os.path.join("Image-Classification/fashion6",img) for img in os.listdir("Image-Classification/fashion6")]

In [ ]:
print(len(image_loc1))
print(len(image_loc2))
print(len(image_loc3))
print(len(image_loc4))
print(len(image_loc5))
print(len(image_loc6))

In [ ]:
##combining all the images in one list
image_loc_list=image_loc1 + image_loc2 + image_loc3 + image_loc4 + image_loc5 + image_loc6
len(image_loc_list)

In [49]:
##reading,processing and saving the image data into image_array list
j=0
image_array=[None]*len(image_loc_list)
for i in image_loc_list:
    image=cv2.imread(i)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=cv2.resize(image,(100,100),cv2.INTER_LINEAR)
    image=image.reshape(100*100)
    image_array[j]=image
    j+=1    

In [53]:
##Creating Dataframe from the list of image data
Pd_mat=pd.DataFrame(image_array)

In [ ]:
##removing any duplicates in the Predictor Matrix
Pd_mat.drop_duplicates()

In [ ]:
##displaying sample image
image_array=np.asarray(image_array)
plt.imshow(image_array[100].reshape(100,100),cmap="gray")